In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.transforms import Resize
import segmentation_models_pytorch as smp

from src.models.BaselineModel import BaselineModel
from src.evaluation.evaluate_result import evaluate_result
from src.datasets.INRIAAerialImageLabellingDataset import (
    INRIAAerialImageLabellingDataset,
)

from src.datasets.utils.ResizeToDivisibleBy32 import ResizeToDivisibleBy32

d:\__repos\aerial_segmentation\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prepare environment

In [2]:
torch.cuda.is_available()

True

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [7]:
VAL_SIZE = 0.2
BATCH_SIZE = 1
SEED = 42
IMAGE_SIZE = 576
INRIA_DATASET_PATH = "data/INRIAAerialImageLabellingDataset"  # home PC
# INRIA_DATASET_PATH = "data/TestSubsets/INRIAAerialImageLabellingDataset"  # laptop

In [8]:
labeled_dataset = INRIAAerialImageLabellingDataset(
    INRIA_DATASET_PATH,
    transforms=[
        Resize(IMAGE_SIZE),
        # ResizeToDivisibleBy32()
    ],
)
print(len(labeled_dataset))

data/INRIAAerialImageLabellingDataset\train
180


In [9]:
test_dataset = INRIAAerialImageLabellingDataset(
    INRIA_DATASET_PATH,
    split="test",
    transforms=[
        Resize(IMAGE_SIZE),
        # ResizeToDivisibleBy32()
    ],
)
print(len(test_dataset))

data/INRIAAerialImageLabellingDataset\test
144


## Sanity check data

In [10]:
train_loader = DataLoader(labeled_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [11]:
for images, masks in train_loader:
    print(images.shape)
    print(masks.shape)
    break

torch.Size([1, 3, 576, 576])
torch.Size([1, 1, 576, 576])


In [12]:
to_pil_transform = transforms.ToPILImage()

In [13]:
img = to_pil_transform(images.squeeze())

In [14]:
# plt.imsave("assets/inria-example-image.jpeg", np.array(img))

In [15]:
# img.show()

In [16]:
msk = to_pil_transform(masks.squeeze())

In [17]:
# plt.imsave("assets/inria-example-mask.jpeg", np.array(msk))

In [18]:
# msk.show()

# UNET

## Run example model on single image

In [19]:
model = smp.Unet(
    encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,  # model output channels (number of classes in your dataset)
).to(device)

In [20]:
baseline_model = BaselineModel()

In [21]:
torch.cuda.empty_cache()

In [22]:
model.eval()
with torch.no_grad():
    for images, masks in train_loader:
        print(images.shape)
        print(masks.shape)
        break
    output = model(images.to(device))
    output = (output > 0.5).float()

    output_baseline = baseline_model(images.to(device))
    output_baseline = (output_baseline > 0.5).float()

torch.Size([1, 3, 576, 576])
torch.Size([1, 1, 576, 576])


In [23]:
print(output.shape)

torch.Size([1, 1, 576, 576])


In [24]:
print(output_baseline.shape)

torch.Size([1, 1, 576, 576])


In [25]:
outp = to_pil_transform(output.squeeze().cpu().numpy())

In [26]:
unique, counts = np.unique(outp, return_counts=True)
print(dict(zip(unique, counts)))

{0: 329368, 255: 2408}


In [27]:
unique, counts = np.unique(output.cpu(), return_counts=True)
print(dict(zip(unique, counts)))

{0.0: 329368, 1.0: 2408}


In [28]:
# outp.show()

In [29]:
rand_outp = to_pil_transform(output_baseline.squeeze().cpu().numpy())

In [30]:
unique, counts = np.unique(rand_outp, return_counts=True)
print(dict(zip(unique, counts)))

{0: 165760, 255: 166016}


In [31]:
# rand_outp.show()

In [32]:
evaluate_result(output, masks)

{'iou': tensor(0.0058),
 'f1': tensor(0.0115),
 'accuracy': tensor(0.9227),
 'recall': tensor(0.0124)}

In [33]:
evaluate_result(output_baseline, masks)

{'iou': tensor(0.0665),
 'f1': tensor(0.1247),
 'accuracy': tensor(0.4999),
 'recall': tensor(0.5045)}

In [34]:
evaluate_result(masks, masks)

{'iou': tensor(1.),
 'f1': tensor(1.),
 'accuracy': tensor(1.),
 'recall': tensor(1.)}

## Run example model on dataset

In [35]:
model.eval()
metrics_dict = {
    "iou": [],
    "f1": [],
    "accuracy": [],
    "recall": [],
}

In [36]:
with torch.no_grad():
    for images, masks in train_loader:
        output = model(images.to(device))
        output = (output > 0.5).float()

        iter_metrics = evaluate_result(output, masks)
        for key in metrics_dict.keys():
            metrics_dict[key].append(iter_metrics[key])

In [37]:
print(metrics_dict)

{'iou': [tensor(0.0076), tensor(0.0028), tensor(0.0123), tensor(0.0083), tensor(0.0092), tensor(0.0060), tensor(0.0058), tensor(0.0043), tensor(0.0108), tensor(0.0087), tensor(0.0071), tensor(0.0072), tensor(0.0099), tensor(0.0068), tensor(0.0081), tensor(0.0210), tensor(0.0040), tensor(0.0052), tensor(0.0068), tensor(0.0078), tensor(0.0091), tensor(0.0004), tensor(0.0073), tensor(0.0100), tensor(0.0066), tensor(0.0066), tensor(0.0164), tensor(0.0086), tensor(0.0147), tensor(0.0144), tensor(0.0095), tensor(0.0060), tensor(0.0362), tensor(0.0023), tensor(0.0034), tensor(0.0056), tensor(0.0082), tensor(0.0052), tensor(0.0062), tensor(0.0026), tensor(0.0041), tensor(0.0113), tensor(0.0033), tensor(0.0059), tensor(0.0084), tensor(0.0074), tensor(0.0058), tensor(0.0140), tensor(0.0081), tensor(0.0158), tensor(0.0093), tensor(0.0055), tensor(0.0002), tensor(0.0041), tensor(0.0131), tensor(0.0088), tensor(0.0161), tensor(0.0036), tensor(0.0038), tensor(0.0154), tensor(0.0091), tensor(0.0041),

In [38]:
print({key: np.mean(value) for key, value in metrics_dict.items()})

{'iou': 0.007533122, 'f1': 0.014907256, 'accuracy': 0.96392655, 'recall': 0.28058752}


# UNET++

## Run example model on single image

In [39]:
model = smp.UnetPlusPlus(
    encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,  # model output channels (number of classes in your dataset)
).to(device)

In [40]:
baseline_model = BaselineModel()

In [41]:
torch.cuda.empty_cache()

In [42]:
model.eval()
with torch.no_grad():
    for images, masks in train_loader:
        print(images.shape)
        print(masks.shape)
        break
    output = model(images.to(device))
    output = (output > 0.5).float()

    output_baseline = baseline_model(images.to(device))
    output_baseline = (output_baseline > 0.5).float()

torch.Size([1, 3, 576, 576])
torch.Size([1, 1, 576, 576])


In [43]:
print(output.shape)

torch.Size([1, 1, 576, 576])


In [44]:
print(output_baseline.shape)

torch.Size([1, 1, 576, 576])


In [45]:
to_pil_transform = transforms.ToPILImage()

In [46]:
img = to_pil_transform(images.squeeze())

In [47]:
plt.imsave("assets/inria-example-image.jpeg", np.array(img))

In [48]:
# img.show()

In [49]:
msk = to_pil_transform(masks.squeeze())

In [50]:
plt.imsave("assets/inria-example-mask.jpeg", np.array(msk))

In [51]:
# msk.show()

In [52]:
outp = to_pil_transform(output.squeeze().cpu())

In [53]:
plt.imsave("assets/inria-example-unet-plus-plus-output.jpeg", np.array(outp))

In [54]:
# outp.show()

In [55]:
evaluate_result(output, masks)

{'iou': tensor(0.),
 'f1': tensor(0.),
 'accuracy': tensor(0.9810),
 'recall': tensor(0.0556)}